In [1]:
# 0) Paths
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..").resolve()
RES_DIR = PROJECT_ROOT / "results"
RES_DIR.mkdir(exist_ok=True, parents=True)

# File Paths
pd_path   = RES_DIR / "stress_pd_summary.csv"      # scenario, family, mean_pd, p50, p90, p99, uplift_vs_baseline_pct
el_path   = RES_DIR / "stress_el_summary.csv"      # scenario, family, mean_pd, EL, EL_change_vs_baseline_pct
macro_all_path = RES_DIR / "macro_scenarios_all.csv"  # data-driven + Fed scenarios (macro)
genai_path     = RES_DIR / "genai_stress_scenarios.csv"      # GenAI macro scenarios

In [2]:
# 1) Load risk tables

pd_df = pd.read_csv(pd_path)
el_df = pd.read_csv(el_path)
macro_all_df = pd.read_csv(macro_all_path)
genai_df = pd.read_csv(genai_path)

# Clean colnames (strip whitespace)
pd_df.columns = [c.strip() for c in pd_df.columns]
el_df.columns = [c.strip() for c in el_df.columns]

print("PD DF columns:", pd_df.columns.tolist())
print("EL DF columns:", el_df.columns.tolist())
print("Macro all DF columns:", macro_all_df.columns.tolist())
print("GenAI DF columns:", genai_df.columns.tolist())


PD DF columns: ['scenario', 'family', 'mean_pd', 'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct']
EL DF columns: ['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct']
Macro all DF columns: ['scenario', 'family', 'GDPC1', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'UNRATE_delta_qoq', 'FEDFUNDS_delta_qoq', 'GDPC1_delta_qoq', 'inflation_qoq', 'real_rate_qoq']
GenAI DF columns: ['scenario', 'GDPC1', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'GDPC1_delta_qoq', 'UNRATE_delta_qoq', 'CPIAUCSL_delta_qoq', 'FEDFUNDS_delta_qoq', 'inflation_qoq', 'real_rate_qoq']


In [3]:
el_df = el_df.drop(columns=["family", "mean_pd"])

In [4]:
# Ensure required columns exist
required_pd = {"scenario", "family", "mean_pd"}
required_el = {"scenario", "EL", "EL_change_vs_baseline_pct"}

missing_pd = required_pd - set(pd_df.columns)
missing_el = required_el - set(el_df.columns)
if missing_pd:
    raise ValueError(f"PD summary is missing columns: {missing_pd}")
if missing_el:
    raise ValueError(f"EL summary is missing columns: {missing_el}")

# Merge PD + EL on (scenario, family, mean_pd)
risk_df = pd.merge(
    pd_df,
    el_df[list(required_el)],  # keep only key + EL info
    on= "scenario",
    how="left",
    validate="one_to_one"
)

print("Merged risk_df shape:", risk_df.shape)
display(risk_df.head())

Merged risk_df shape: (16, 9)


,scenario,family,mean_pd,p50_pd,p90_pd,p99_pd,uplift_vs_baseline_pct,EL,EL_change_vs_baseline_pct
0,baseline_actual,baseline,0.205647,0.176045,0.392747,0.618904,0.000000,7.323655e+08,0.000000
1,mild_adverse,data_driven,0.265297,0.255287,0.420288,0.568975,29.006001,9.447954e+08,29.006001
2,severe_adverse,data_driven,0.154359,0.142171,0.259541,0.389576,-24.939762,5.497153e+08,-24.939762
3,Fed_Adverse,Fed,0.023480,0.020046,0.041469,0.073020,-88.582595,8.361713e+07,-88.582595
4,Fed_Baseline,Fed,0.243041,0.232050,0.389895,0.537805,18.183698,8.655366e+08,18.183698


In [5]:
# 2) Load macro scenario tables (Fed + data-driven + GenAI)

macro_df_list = []

# --- Load FED + DATA-DRIVEN macro scenarios ---
if macro_all_path.exists():
    macro_all = pd.read_csv(macro_all_path)
    macro_all.columns = macro_all.columns.str.strip()

    # Keep only macro features + scenario
    fed_macro_cols = [
        "scenario", "GDPC1", "UNRATE", "CPIAUCSL", "FEDFUNDS",
        "UNRATE_delta_qoq", "FEDFUNDS_delta_qoq",
        "GDPC1_delta_qoq", "inflation_qoq", "real_rate_qoq"
    ]
    macro_all = macro_all[fed_macro_cols]

    macro_all["macro_source"] = "macro_all"
    macro_df_list.append(macro_all)

# --- Load GenAI macro scenarios ---
if genai_path.exists():
    genai_macro = pd.read_csv(genai_path)
    genai_macro.columns = genai_macro.columns.str.strip()

    # Some GenAI tables include CPIAUCSL_delta_qoq; keep if exists
    genai_keep = [c for c in [
        "scenario", "GDPC1", "UNRATE", "CPIAUCSL", "FEDFUNDS",
        "UNRATE_delta_qoq", "FEDFUNDS_delta_qoq",
        "GDPC1_delta_qoq", "inflation_qoq", "real_rate_qoq",
        "CPIAUCSL_delta_qoq"  # may or may not exist
    ] if c in genai_macro.columns]

    genai_macro = genai_macro[genai_keep]
    genai_macro["macro_source"] = "GenAI"
    macro_df_list.append(genai_macro)

# === Clean + merge ===
if macro_df_list:
    # Combine all macro tables
    macro_df = pd.concat(macro_df_list, ignore_index=True)

    if "scenario" not in macro_df.columns:
        raise ValueError("Macro scenario tables must contain a 'scenario' column.")

    # If scenario appears both in macro_all and genAI → keep macro_all version
    macro_df = (
        macro_df.sort_values("macro_source")
                .drop_duplicates(subset=["scenario"], keep="first")
                .reset_index(drop=True)
    )

    # Identify macro feature columns
    macro_cols = [
        c for c in macro_df.columns
        if c not in {"scenario", "macro_source"}
    ]

    print("Macro columns:", macro_cols)

    # Final merge risk + macro
    full_df = pd.merge(
        risk_df,                          # PD + EL
        macro_df[["scenario"] + macro_cols], 
        on="scenario",
        how="left",
        validate="one_to_one"
    )

else:
    macro_cols = []
    full_df = risk_df.copy()
    print("No macro scenario files found; JSON will have empty 'macro' blocks.")

print("Final merged full_df shape:", full_df.shape)
display(full_df.head())


Macro columns: ['GDPC1', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'UNRATE_delta_qoq', 'FEDFUNDS_delta_qoq', 'GDPC1_delta_qoq', 'inflation_qoq', 'real_rate_qoq', 'CPIAUCSL_delta_qoq']
Final merged full_df shape: (16, 19)


,scenario,family,mean_pd,p50_pd,p90_pd,p99_pd,uplift_vs_baseline_pct,EL,EL_change_vs_baseline_pct,GDPC1,UNRATE,CPIAUCSL,FEDFUNDS,UNRATE_delta_qoq,FEDFUNDS_delta_qoq,GDPC1_delta_qoq,inflation_qoq,real_rate_qoq,CPIAUCSL_delta_qoq
0,baseline_actual,baseline,0.205647,0.176045,0.392747,0.618904,0.000000,7.323655e+08,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mild_adverse,data_driven,0.265297,0.255287,0.420288,0.568975,29.006001,9.447954e+08,29.006001,18782.243000,5.433333,247.238333,1.203333,-0.013072,0.202216,0.004002,0.006348,0.154176,NaN
2,severe_adverse,data_driven,0.154359,0.142171,0.259541,0.389576,-24.939762,5.497153e+08,-24.939762,17953.974000,6.933333,251.686333,1.923333,0.006803,0.555556,0.001845,0.007955,0.190550,NaN
3,Fed_Adverse,Fed,0.023480,0.020046,0.041469,0.073020,-88.582595,8.361713e+07,-88.582595,18821.441681,5.475000,257.030458,0.225000,0.800000,-0.500000,-0.035000,0.013000,-0.513000,NaN
4,Fed_Baseline,Fed,0.243041,0.232050,0.389895,0.537805,18.183698,8.655366e+08,18.183698,21214.070968,3.925000,260.107147,1.775000,0.000000,0.300000,0.028000,0.019000,0.281000,NaN


In [6]:

# 3) Build LLM-ready JSON

scenario_records = []

for _, row in full_df.iterrows():
    scen_name = str(row["scenario"])
    family    = str(row["family"])

    # ---- risk block ----
    risk_block = {
        "mean_pd": float(row["mean_pd"]),
    }
    # Optional PD distribution metrics if present
    for c in ["p50_pd", "p90_pd", "p99_pd", "uplift_vs_baseline_pct"]:
        if c in full_df.columns and pd.notna(row.get(c, None)):
            risk_block[c] = float(row[c])

    # EL info from EL summary
    if "EL" in full_df.columns and pd.notna(row.get("EL", None)):
        risk_block["EL"] = float(row["EL"])
    if "EL_change_vs_baseline_pct" in full_df.columns and pd.notna(row.get("EL_change_vs_baseline_pct", None)):
        risk_block["EL_change_vs_baseline_pct"] = float(row["EL_change_vs_baseline_pct"])

    # ---- macro block ----
    macro_block = {}
    for c in macro_cols:
        val = row.get(c, None)
        if pd.notna(val):
            # convert numpy -> Python scalar
            try:
                macro_block[c] = float(val)
            except Exception:
                macro_block[c] = val

    record = {
        "scenario": scen_name,
        "family": family,
        "macro": macro_block,
        "risk": risk_block,
    }
    scenario_records.append(record)


In [7]:
scenario_records

[{'scenario': 'baseline_actual',
  'family': 'baseline',
  'macro': {},
  'risk': {'mean_pd': 0.2056470562490912,
   'p50_pd': 0.1760445491639388,
   'p90_pd': 0.3927474678281696,
   'p99_pd': 0.618903696301832,
   'uplift_vs_baseline_pct': 0.0,
   'EL': 732365482.4640737,
   'EL_change_vs_baseline_pct': 0.0}},
 {'scenario': 'mild_adverse',
  'family': 'data_driven',
  'macro': {'GDPC1': 18782.243,
   'UNRATE': 5.433333333333334,
   'CPIAUCSL': 247.23833333333337,
   'FEDFUNDS': 1.2033333333333334,
   'UNRATE_delta_qoq': -0.0130718954248367,
   'FEDFUNDS_delta_qoq': 0.2022160664819943,
   'GDPC1_delta_qoq': 0.0040024506125282,
   'inflation_qoq': 0.0063478253641482,
   'real_rate_qoq': 0.1541756763574446},
  'risk': {'mean_pd': 0.2652970439388467,
   'p50_pd': 0.2552873581312398,
   'p90_pd': 0.4202883156767631,
   'p99_pd': 0.5689753416407071,
   'uplift_vs_baseline_pct': 29.00600124200372,
   'EL': 944795423.4036094,
   'EL_change_vs_baseline_pct': 29.00600124200372}},
 {'scenario': 

In [8]:

# 4) Save JSON + flat CSV
import json
json_path = RES_DIR / "llm_scenarios.json"
flat_csv_path = RES_DIR / "llm_scenarios_flat.csv"

with json_path.open("w", encoding="utf-8") as f:
    json.dump(scenario_records, f, ensure_ascii=False, indent=2)

# Flatten for CSV inspection
flat_rows = []
for rec in scenario_records:
    base = {
        "scenario": rec["scenario"],
        "family": rec["family"],
        "mean_pd": rec["risk"].get("mean_pd"),
        "EL": rec["risk"].get("EL"),
        "EL_change_vs_baseline_pct": rec["risk"].get("EL_change_vs_baseline_pct"),
        "p50_pd": rec["risk"].get("p50_pd"),
        "p90_pd": rec["risk"].get("p90_pd"),
        "p99_pd": rec["risk"].get("p99_pd"),
        "uplift_vs_baseline_pct": rec["risk"].get("uplift_vs_baseline_pct"),
    }
    # Add macro_* columns
    for k, v in rec["macro"].items():
        base[f"macro_{k}"] = v
    flat_rows.append(base)

flat_df = pd.DataFrame(flat_rows)
flat_df.to_csv(flat_csv_path, index=False)

print(f"   Prepared {len(scenario_records)} scenarios for LLM.")


   Prepared 16 scenarios for LLM.


In [9]:
flat_df

,scenario,family,mean_pd,EL,EL_change_vs_baseline_pct,p50_pd,p90_pd,p99_pd,uplift_vs_baseline_pct,macro_GDPC1,macro_UNRATE,macro_CPIAUCSL,macro_FEDFUNDS,macro_UNRATE_delta_qoq,macro_FEDFUNDS_delta_qoq,macro_GDPC1_delta_qoq,macro_inflation_qoq,macro_real_rate_qoq,macro_CPIAUCSL_delta_qoq
0,baseline_actual,baseline,0.205647,7.323655e+08,0.000000,0.176045,0.392747,0.618904,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mild_adverse,data_driven,0.265297,9.447954e+08,29.006001,0.255287,0.420288,0.568975,29.006001,18782.243000,5.433333,247.238333,1.203333,-0.013072,0.202216,0.004002,0.006348,0.154176,NaN
2,severe_adverse,data_driven,0.154359,5.497153e+08,-24.939762,0.142171,0.259541,0.389576,-24.939762,17953.974000,6.933333,251.686333,1.923333,0.006803,0.555556,0.001845,0.007955,0.190550,NaN
3,Fed_Adverse,Fed,0.023480,8.361713e+07,-88.582595,0.020046,0.041469,0.073020,-88.582595,18821.441681,5.475000,257.030458,0.225000,0.800000,-0.500000,-0.035000,0.013000,-0.513000,NaN
4,Fed_Baseline,Fed,0.243041,8.655366e+08,18.183698,0.232050,0.389895,0.537805,18.183698,21214.070968,3.925000,260.107147,1.775000,0.000000,0.300000,0.028000,0.019000,0.281000,NaN
5,Fed_Severe,Fed,0.248648,8.855033e+08,20.910030,0.237881,0.397639,0.545858,20.910030,16907.187783,6.900000,254.741601,0.100000,1.500000,0.000000,-0.089000,0.009000,-0.009000,NaN
6,GenAI_S1,GenAI,0.256837,9.146661e+08,24.892028,0.246426,0.408844,0.557375,24.892028,18121.750000,6.790451,232.722763,0.400428,-0.013682,-0.034408,0.005396,0.004757,-0.039166,0.004757
7,GenAI_S2,GenAI,0.260649,9.282416e+08,26.745685,0.250415,0.414017,0.562639,26.745685,18117.689453,6.792233,232.722672,0.396302,-0.011550,-0.030678,0.005145,0.004872,-0.035550,0.004872
8,GenAI_S3,GenAI,0.274309,9.768911e+08,33.388464,0.264763,0.432333,0.581011,33.388464,18107.136719,6.807709,232.481750,0.377757,-0.004043,-0.021604,0.005606,0.005628,-0.027232,0.005628
9,GenAI_S4,GenAI,0.262962,9.364797e+08,27.870535,0.252838,0.417143,0.565803,27.870535,18099.484375,6.788488,232.671570,0.387199,-0.012064,-0.027244,0.003164,0.003727,-0.030971,0.003727


In [10]:
# === 0) Imports & setup ==========================================
from pathlib import Path
import json

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

# Assume flat_df already exists from your earlier steps
# flat_df columns (example):
# ['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct',
#  'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct',
#  'macro_GDPC1', 'macro_UNRATE', 'macro_CPIAUCSL', 'macro_FEDFUNDS', ... ]

RES_DIR = Path("../results")
llm_path = RES_DIR / "llm_narratives.json"

# Work on a copy so we don't mutate flat_df accidentally
merged_df = flat_df.copy()

print("Initial merged_df columns:")
print(merged_df.columns.tolist())


Initial merged_df columns:
['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct', 'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct', 'macro_GDPC1', 'macro_UNRATE', 'macro_CPIAUCSL', 'macro_FEDFUNDS', 'macro_UNRATE_delta_qoq', 'macro_FEDFUNDS_delta_qoq', 'macro_GDPC1_delta_qoq', 'macro_inflation_qoq', 'macro_real_rate_qoq', 'macro_CPIAUCSL_delta_qoq']


In [11]:
# === 1) Load LLM narratives JSON and merge =======================

if llm_path.exists():
    with open(llm_path, "r") as f:
        llm_data = json.load(f)

    llm_df = pd.DataFrame(llm_data)

    # Align columns with our scenario table
    # JSON example:
    # {
    #   "scenario_name": "...",
    #   "headline": "...",
    #   "macro_story": "...",
    #   "credit_risk_impact": "...",
    #   "comparison_to_baseline": "...",
    #   "key_risks": [...],
    #   "management_actions": [...],
    #   "tone": "benign",
    #   "scenario_family": "baseline"
    # }
    llm_df = llm_df.rename(
        columns={
            "scenario_name": "scenario",
            "scenario_family": "llm_family"
        }
    )

    # Keep only the narrative-related columns
    narrative_cols = [
        "scenario",
        "headline",
        "macro_story",
        "credit_risk_impact",
        "comparison_to_baseline",
        "key_risks",
        "management_actions",
        "tone",
        "llm_family",
    ]
    # Some fields may not exist depending on how you generated JSON
    narrative_cols = [c for c in narrative_cols if c in llm_df.columns]

    merged_df = merged_df.merge(
        llm_df[narrative_cols],
        on="scenario",
        how="left",
        validate="one_to_one"
    )

    print("LLM narratives merged. Columns now:")
    print(merged_df.columns.tolist())
else:
    print(f"LLM narrative file not found at: {llm_path}")


LLM narrative file not found at: ../results/llm_narratives.json


In [12]:
# === 2) Helper: pretty printer for a single scenario =============

def show_scenario(name: str):
    """Render one scenario (macro + risk + LLM narrative) as Markdown."""
    r = merged_df.loc[merged_df["scenario"] == name]
    if r.empty:
        print(f"Scenario '{name}' not found in merged_df.")
        return
    r = r.iloc[0]

    md = f"## Scenario: **{r['scenario']}**  \n"
    md += f"Family: **{r['family']}**  \n\n"

    # --- Macro block ------------------------------------------------
    macro_cols = [c for c in merged_df.columns if c.startswith("macro_")]
    if macro_cols:
        md += "### Macro Environment\n"
        for col in macro_cols:
            val = r.get(col, None)
            if pd.notna(val):
                raw_name = col.replace("macro_", "")
                if isinstance(val, (int, float, np.number)):
                    md += f"- **{raw_name}**: {float(val):,.4f}\n"
                else:
                    md += f"- **{raw_name}**: {val}\n"
        md += "\n"

    # --- Risk metrics block -----------------------------------------
    md += "### Risk Metrics\n"
    risk_cols = [
        "mean_pd",
        "p50_pd", "p90_pd", "p99_pd",
        "EL", "EL_change_vs_baseline_pct",
        "uplift_vs_baseline_pct",
    ]
    for col in risk_cols:
        if col in merged_df.columns and pd.notna(r.get(col, None)):
            val = r[col]
            if isinstance(val, (int, float, np.number)):
                md += f"- **{col}**: {float(val):,.4f}\n"
            else:
                md += f"- **{col}**: {val}\n"
    md += "\n"

    # --- LLM narrative block ----------------------------------------
    if "headline" in merged_df.columns and pd.notna(r.get("headline", None)):
        md += "### LLM Narrative\n"
        md += f"**Headline:** {r['headline']}\n\n"

        if pd.notna(r.get("macro_story", None)):
            md += f"**Macro story.** {r['macro_story']}\n\n"

        if pd.notna(r.get("credit_risk_impact", None)):
            md += f"**Credit risk impact.** {r['credit_risk_impact']}\n\n"

        if pd.notna(r.get("comparison_to_baseline", None)):
            md += f"**Comparison to baseline.** {r['comparison_to_baseline']}\n\n"

        # key_risks and management_actions might be list *or* string
        kr = r.get("key_risks", None)
        if isinstance(kr, list):
            md += "**Key risks:**\n"
            for k in kr:
                md += f"- {k}\n"
            md += "\n"
        elif isinstance(kr, str) and kr.strip():
            md += f"**Key risks:** {kr}\n\n"

        ma = r.get("management_actions", None)
        if isinstance(ma, list):
            md += "**Suggested management actions:**\n"
            for a in ma:
                md += f"- {a}\n"
            md += "\n"
        elif isinstance(ma, str) and ma.strip():
            md += f"**Suggested management actions:** {ma}\n\n"

        if pd.notna(r.get("tone", None)):
            md += f"_Narrative tone: **{r['tone']}**._\n"
    else:
        md += "### LLM Narrative\n"
        md += "*(No narrative attached — run `llm_narratives.py` or check the JSON fields.)*\n"

    display(Markdown(md))


In [13]:
# === 5) Show examples ===
show_scenario("baseline_actual")


## Scenario: **baseline_actual**  
Family: **baseline**  

### Macro Environment

### Risk Metrics
- **mean_pd**: 0.2056
- **p50_pd**: 0.1760
- **p90_pd**: 0.3927
- **p99_pd**: 0.6189
- **EL**: 732,365,482.4641
- **EL_change_vs_baseline_pct**: 0.0000
- **uplift_vs_baseline_pct**: 0.0000

### LLM Narrative
*(No narrative attached — run `llm_narratives.py` or check the JSON fields.)*


In [14]:
show_scenario("mild_adverse")

## Scenario: **mild_adverse**  
Family: **data_driven**  

### Macro Environment
- **GDPC1**: 18,782.2430
- **UNRATE**: 5.4333
- **CPIAUCSL**: 247.2383
- **FEDFUNDS**: 1.2033
- **UNRATE_delta_qoq**: -0.0131
- **FEDFUNDS_delta_qoq**: 0.2022
- **GDPC1_delta_qoq**: 0.0040
- **inflation_qoq**: 0.0063
- **real_rate_qoq**: 0.1542

### Risk Metrics
- **mean_pd**: 0.2653
- **p50_pd**: 0.2553
- **p90_pd**: 0.4203
- **p99_pd**: 0.5690
- **EL**: 944,795,423.4036
- **EL_change_vs_baseline_pct**: 29.0060
- **uplift_vs_baseline_pct**: 29.0060

### LLM Narrative
*(No narrative attached — run `llm_narratives.py` or check the JSON fields.)*


In [15]:
show_scenario("Fed_Severe")

## Scenario: **Fed_Severe**  
Family: **Fed**  

### Macro Environment
- **GDPC1**: 16,907.1878
- **UNRATE**: 6.9000
- **CPIAUCSL**: 254.7416
- **FEDFUNDS**: 0.1000
- **UNRATE_delta_qoq**: 1.5000
- **FEDFUNDS_delta_qoq**: 0.0000
- **GDPC1_delta_qoq**: -0.0890
- **inflation_qoq**: 0.0090
- **real_rate_qoq**: -0.0090

### Risk Metrics
- **mean_pd**: 0.2486
- **p50_pd**: 0.2379
- **p90_pd**: 0.3976
- **p99_pd**: 0.5459
- **EL**: 885,503,326.5758
- **EL_change_vs_baseline_pct**: 20.9100
- **uplift_vs_baseline_pct**: 20.9100

### LLM Narrative
*(No narrative attached — run `llm_narratives.py` or check the JSON fields.)*


In [16]:
show_scenario("GenAI_S7")

## Scenario: **GenAI_S7**  
Family: **GenAI**  

### Macro Environment
- **GDPC1**: 18,077.7129
- **UNRATE**: 6.7994
- **CPIAUCSL**: 232.3836
- **FEDFUNDS**: 0.4127
- **UNRATE_delta_qoq**: -0.0073
- **FEDFUNDS_delta_qoq**: 0.0801
- **GDPC1_delta_qoq**: 0.0023
- **inflation_qoq**: 0.0034
- **real_rate_qoq**: 0.0767
- **CPIAUCSL_delta_qoq**: 0.0034

### Risk Metrics
- **mean_pd**: 0.3649
- **p50_pd**: 0.3617
- **p90_pd**: 0.5452
- **p99_pd**: 0.6858
- **EL**: 1,299,531,421.3471
- **EL_change_vs_baseline_pct**: 77.4430
- **uplift_vs_baseline_pct**: 77.4430

### LLM Narrative
*(No narrative attached — run `llm_narratives.py` or check the JSON fields.)*


In [17]:
show_scenario("GenAI_S8")

## Scenario: **GenAI_S8**  
Family: **GenAI**  

### Macro Environment
- **GDPC1**: 18,078.8984
- **UNRATE**: 6.7733
- **CPIAUCSL**: 232.5001
- **FEDFUNDS**: 0.4237
- **UNRATE_delta_qoq**: -0.0074
- **FEDFUNDS_delta_qoq**: 0.0616
- **GDPC1_delta_qoq**: 0.0026
- **inflation_qoq**: 0.0016
- **real_rate_qoq**: 0.0600
- **CPIAUCSL_delta_qoq**: 0.0016

### Risk Metrics
- **mean_pd**: 0.3428
- **p50_pd**: 0.3378
- **p90_pd**: 0.5190
- **p99_pd**: 0.6627
- **EL**: 1,220,894,460.6462
- **EL_change_vs_baseline_pct**: 66.7056
- **uplift_vs_baseline_pct**: 66.7056

### LLM Narrative
*(No narrative attached — run `llm_narratives.py` or check the JSON fields.)*
